In [1]:
import pandas as pd
from numpy.linalg import norm
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import ShuffleSplit

In [34]:
movie_genre = pd.read_csv('../hw3_data/Movielens/movie_genre.dat', sep='\t', header=None)
movie_movie = pd.read_csv('../hw3_data/Movielens/movie_movie(knn).dat', sep='\t', header=None)
user_age = pd.read_csv('../hw3_data/Movielens/user_age.dat', sep='\t', header=None)
user_movie = pd.read_csv('../hw3_data/Movielens/user_movie.dat', sep='\t', header=None)
user_occupation = pd.read_csv('../hw3_data/Movielens/user_occupation.dat', sep='\t', header=None)
user_user = pd.read_csv('../hw3_data/Movielens/user_user(knn).dat', sep='\t', header=None)

In [35]:
user_interaction = user_movie[1].value_counts()
user_movie = user_movie[user_movie[1].isin(user_interaction.index[user_interaction.gt(3)])]
user_movie.columns = ['user', 'movie', 'rating', 'timestamp']

In [93]:
movie_genre = np.load('../hw3_data/Movielens/movie_genre.npy')
movie_movie = np.load('../hw3_data/Movielens/movie_movie.npy')
user_age = np.load('../hw3_data/Movielens/user_age.npy')
user_movie = np.load('../hw3_data/Movielens/user_movie.npy')
user_occupation = np.load('../hw3_data/Movielens/user_occupation.npy')
user_user = np.load('../hw3_data/Movielens/user_user.npy')

X = user_movie[:,:2]
y = user_movie[:,-1]
entity_user = 943
entity_item = 1682
age, occupation, genre = 8, 21, 18 # entity amount 

In [94]:
user_movie.shape

(99543, 3)

In [58]:
data = X[0]
user, item = data

ua = user_age[user_age[:,0] == user][:,1][0]
uo = user_occupation[user_occupation[:,0] == user][:,1][0]
train_example = np.array([user,item,ua,uo])
train_example

(196, 242, 5, 3)


array([196, 242,   5,   3])

In [91]:
mg = movie_genre[movie_genre[:,0] == 267][:,1]
mg = "|".join(str(x) for x in mg)
if mg =='':
    mg = str(0)
mg

'0'

In [96]:
def get_embedding(data, ):
    user, item = data
    
    ua = user_age[user_age[:,0] == user][:,1][0]
    uo = user_occupation[user_occupation[:,0] == user][:,1][0]
    mg = movie_genre[movie_genre[:,0] == item][:,1]
    mg = "|".join(str(x) for x in mg)
    if mg =='':
        mg = str(0)
    # fea_genre = np.zeros((1, genre), dtype=int)
    # mg = movie_genre[movie_genre[:,0] == item][:,1]
    # fea_genre[0, mg-1] = 1

    # user_knn = np.zeros((1, entity_user), dtype=int)
    # uu = user_user[user_user[:,0] == user][:,1].astype(int)
    # user_knn[0, uu-1] = 1

    # item_knn = np.zeros((1, entity_item), dtype=int)
    # mm = movie_movie[movie_movie[:,0] == item][:,1].astype(int)
    # item_knn[0, mm-1] = 1
    
    # train_example = np.concatenate((user_index, item_index, fea_age, fea_occupation, fea_genre, user_knn, item_knn), axis=1)
    train_example = np.array([user,item,ua,uo,mg])
    return train_example


embed_X = np.apply_along_axis(get_embedding, 1, X)
embed = np.concatenate((embed_X, y.reshape(-1,1)),axis =1)

In [97]:
embed = pd.DataFrame(embed, columns=['user', 'movie', 'age', 'occupation', 'genre','rating'])
embed

,user,movie,age,occupation,genre,rating
0,196,242,5,3,5,3
1,186,302,4,4,6|10|13|16,3
2,22,377,3,3,4|5,1
3,244,51,3,1,8|14|17|18,2
4,166,346,5,8,6|8,1
...,...,...,...,...,...,...
99538,880,476,2,6,5,3
99539,716,204,4,5,5|15,5
99540,276,1090,3,6,16,1
99541,13,225,5,8,4|5,2


In [98]:
embed.to_csv('../hw3_data/Movielens/movielens_sample.csv',index=False)

In [36]:
def concate_age(data):
    user = data['user']
    ua = user_age[user_age[0] == user][1]
    uo = user_occupation[user_occupation[0] == user][1]
    return ua,uo

# def concate_occupation(data):
#     user, item = data['user'], data['movie']
#     uo = user_occupation[user_occupation[0] == user][1]
#     return uo

In [38]:
user_movie['age'] = user_movie.apply(concate_age, axis=1)
# user_movie['occupation'] = user_movie.apply(concate_occupation , axis=1)